# 初期化

In [4]:
!pip install fastlabel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.1/80.1 kB 2.4 MB/s eta 0:00:00
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.8/114.8 kB 3.0 MB/s eta 0:00:00
  Using cached geojson-3.1.0-py3-none-any.whl.metadata (16 kB)
  Using cached xmltodict-0.12.0-py2.py3-none-any.whl.metadata (6.3 kB)
  Using cached opencv_python-4.10.0.84-cp37-abi3-macosx_11_0_arm64.whl.metadata (20 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached urllib3-2.2.2-py3-none-any.whl.metadata (6.4 kB)
  Using cached certifi-2024.7.4-py3-none-any.whl.metadata (2.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 3.9 MB/s eta 0:00:00
Using cached xmltodict-0.12.0-py2.py3-none-any.whl (9.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.0/389.0 kB 6.1 MB/s eta 0:00:00a 0:00:01
Using cached geojson-3.1.0-py3-none-any.whl (15 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 5.3 MB/s eta 0:00:

# SDK  Example codes


## 1. プロジェクトの作成とタスクの追加

#### 使い方
1. project_slugが"fastlabel-sample"のプロジェクトが既にある場合、削除する
2. パラメータのFASTLABEL_ACCESS_TOKENを設定
3. 実行

#### パラメータ
1. FASTLABEL_ACCESS_TOKEN:　プラットフォーム上で発行するアクセストークン

In [8]:
import fastlabel
import os

FASTLABEL_ACCESS_TOKEN = "sample-token" # Replace with your access token
PROJECT_NAME = "fastlabel-sample" # Don't need to change this
PROJECT_SLUG = "fastlabel-sample" # Don't need to change this

# Initialize client
os.environ["FASTLABEL_ACCESS_TOKEN"] = FASTLABEL_ACCESS_TOKEN
client = fastlabel.Client()

# Create a project
project = client.create_project(type="image_bbox", name=PROJECT_NAME, slug=PROJECT_SLUG)

# Create tasks
tasks = [
    {"name": "sample1.jpg", "file_path": "./sample_data/images/sample1.jpg"},
    {"name": "sample2.jpg", "file_path": "./sample_data/images/sample2.jpg"},
]

for task in tasks:
    task_id = client.create_image_task(
        project=PROJECT_SLUG,
        name=task["name"],
        file_path=task["file_path"],
    )


## 2. データセットの作成とモデル学習の実行

#### 使い方
1. 名前が"fastlabel-sample"のデータセットが既にある場合、削除する
2. パラメータのFASTLABEL_ACCESS_TOKENを設定
3. 実行

#### パラメータ
1. FASTLABEL_ACCESS_TOKEN:　プラットフォーム上で発行するアクセストークン

In [9]:
import fastlabel
import json
import os

FASTLABEL_ACCESS_TOKEN = "sample-token" # Replace with your access token
DATASET_NAME = "fastlabel-sample" # Don't need to change this

# Initialize client
os.environ["FASTLABEL_ACCESS_TOKEN"] = FASTLABEL_ACCESS_TOKEN
client = fastlabel.Client()

# create dataset
dataset = client.create_dataset(
    name=DATASET_NAME,
)

# load annotations
def remove_unnecessary_keys(annotations):
    keys_to_remove = ["id", "color"]
    clean_annotation = [
        {k: v for k, v in annotation.items() if k not in keys_to_remove}
        for annotation in annotations
    ]
    return clean_annotation


with open("./sample_data/annotation/fastlabel/annotations.json", "r") as f:
    annotations = json.load(f)
    name_annotation_map = {}
    for annotation in annotations:
        name = annotation["name"]
        name_annotation_map[name] = remove_unnecessary_keys(annotation["annotations"])

# create dataset objects
objects = [
    {"name": "sample1.jpg", "file_path": "./sample_data/images/sample1.jpg"},
    {"name": "sample2.jpg", "file_path": "./sample_data/images/sample2.jpg"},
]

for object in objects:
    dataset_object = client.create_dataset_object(
        dataset=DATASET_NAME,
        name=object["name"],
        file_path=object["file_path"],
        tags=[],  # max 5 tags per dataset object.
        licenses=["MIT"],  # max 10 licenses per dataset object
        annotations=name_annotation_map[object["name"]],
    )

# train model
training_job = client.execute_training_job(
    dataset_name=DATASET_NAME,
    base_model_name="fastlabel_object_detection_light",
    epoch=300,
)


## 3. COCO形式のファイルからプロジェクトを作成

#### 使い方
1. 名前が"fastlabel-sample"のプロジェクトが既にある場合、削除する
2. パラメータのFASTLABEL_ACCESS_TOKENを設定
3. 実行

#### パラメータ
1. FASTLABEL_ACCESS_TOKEN:　プラットフォーム上で発行するアクセストークン

In [10]:
import fastlabel
import os

FASTLABEL_ACCESS_TOKEN = "sample-token" # Replace with your access token
PROJECT_NAME = "fastlabel-sample" # Don't need to change this
PROJECT_SLUG = "fastlabel-sample" # Don't need to change this
ANNOTATION_TYPE = "polygon" # Don't need to change this

# Initialize client
os.environ["FASTLABEL_ACCESS_TOKEN"] = FASTLABEL_ACCESS_TOKEN
client = fastlabel.Client()

# Create a project
project = client.create_project(
    type="image_polygon", name=PROJECT_NAME, slug=PROJECT_SLUG
)

# Read annotations from COCO format
annotations_map = client.convert_coco_to_fastlabel(
    file_path="./sample_data/annotation/coco/annotations.json",
    annotation_type=ANNOTATION_TYPE,
)

# Create annotation class
value_set = set()
for key in annotations_map.keys():
    for annotation in annotations_map[key]:
        value_set.add(annotation["value"])
for value in value_set:
    annotation_id = client.create_annotation(
        project=PROJECT_SLUG, type=ANNOTATION_TYPE, value=value, title=value
    )

# Create tasks
tasks = [
    {"name": "sample1.jpg", "file_path": "./sample_data/images/sample1.jpg"},
    {"name": "sample2.jpg", "file_path": "./sample_data/images/sample2.jpg"},
]

for task in tasks:
    task_id = client.create_image_task(
        project=PROJECT_SLUG,
        name=task["name"],
        file_path=task["file_path"],
        annotations=annotations_map.get(task["name"]),
    )


## 4. プロジェクトからCOCO形式でエクスポート

#### 使い方
1. タスクのアノテーションが登録されたプロジェクトを用意する
2. パラメータのFASTLABEL_ACCESS_TOKENを設定
3. パラメータのPROJECT_SLUGに用意したプロジェクトのslugを設定
4. 実行(outputディレクトリが作成されます)

#### パラメータ
1. FASTLABEL_ACCESS_TOKEN:　プラットフォーム上で発行するアクセストークン
2. PROJECT_SLUG:　プロジェクトのスラッグ

In [11]:
import fastlabel
import os

FASTLABEL_ACCESS_TOKEN = "sample-token" # Replace with your access token
PROJECT_SLUG = "fastlabel-sample" # Replace with your project slug

# Initialize client
os.environ["FASTLABEL_ACCESS_TOKEN"] = FASTLABEL_ACCESS_TOKEN
client = fastlabel.Client()

# export coco format
tasks = client.get_image_tasks(project=PROJECT_SLUG)
client.export_coco(project=PROJECT_SLUG, tasks=tasks)


## 5. instance/semanticのマスク画像を出力

#### 使い方
1. タスクとアノテーションが登録されたセグメンテーションプロジェクトを用意する
2. パラメータのFASTLABEL_ACCESS_TOKENを設定
3. パラメータのPROJECT_SLUGに用意したプロジェクトのslugを設定
4. 実行(outputディレクトリが作成されます)

#### パラメータ
1. FASTLABEL_ACCESS_TOKEN:　プラットフォーム上で発行するアクセストークン
2. PROJECT_SLUG:　プロジェクトのスラッグ

In [ ]:
import fastlabel
import os

FASTLABEL_ACCESS_TOKEN = "sample-token" # Replace with your access token
PROJECT_SLUG = "fastlabel-sample" # Replace with your project slug

# Initialize client
os.environ["FASTLABEL_ACCESS_TOKEN"] = FASTLABEL_ACCESS_TOKEN
client = fastlabel.Client()

# export segmentation file
tasks = client.get_image_tasks(project=PROJECT_SLUG)
client.export_instance_segmentation(tasks)

tasks = client.get_image_tasks(project=PROJECT_SLUG)
client.export_semantic_segmentation(tasks)


## 6. 画像プロジェクトの全てのタスクを取得

#### 使い方
1. タスクが登録された画像プロジェクトを用意する
2. パラメータのFASTLABEL_ACCESS_TOKENを設定
3. パラメータのPROJECT_SLUGに用意したプロジェクトのslugを設定
4. 実行

#### パラメータ
1. FASTLABEL_ACCESS_TOKEN:　プラットフォーム上で発行するアクセストークン
2. PROJECT_SLUG:　プロジェクトのスラッグ

In [ ]:
import fastlabel
import os

FASTLABEL_ACCESS_TOKEN = "sample-token" # Replace with your access token
PROJECT_SLUG = "fastlabel-sample" # Replace with your project slug

# Initialize client
os.environ["FASTLABEL_ACCESS_TOKEN"] = FASTLABEL_ACCESS_TOKEN
client = fastlabel.Client()


def get_all_tasks(
    project_slug: str, status: str = None, external_status: str = None, tags: list = []
) -> list:

    # Iterate pages until new tasks are empty.
    result = []
    offset = None

    print("Fetching tasks...")
    while True:
        tasks = client.get_image_tasks(
            # Change to another method (e.g., get_video_tasks) based on your requirement.
            project=project_slug,
            status=status,
            external_status=external_status,
            tags=tags,
            limit=100,
            offset=offset,
        )
        result.extend(tasks)

        if len(tasks) > 0:
            offset = len(result)
            print(offset, " fetched")
        else:
            break

    print("Fetching tasks finished!")
    print("Total tasks count: ", len(result))
    return result


if __name__ == "__main__":
    print(get_all_tasks(project_slug=PROJECT_SLUG))


Fetching tasks...
1  fetched
Fetching tasks finished!
Total tasks count:  1
[{'id': '059d2ec7-3b79-4bb6-b999-5a1bc580dba2', 'name': 'animal_mark01_buta.jpg', 'status': 'registered', 'externalStatus': 'registered', 'url': 'https://annotations.fastlabel.ai/workspaces/7a38045c-71a9-4422-8184-f3c723af7e52/c63325b6-09e6-4fbd-9b1a-d12dcd3fe2d6/tasks/animal_mark01_buta.jpg?updatedAt=1723535926954&Expires=1723680486&Key-Pair-Id=K25YS2907X9Z7Z&Signature=Rq4Y4~g6EsOvSSNWzJKWPw5tmsCQ4EiaO062e-MYFnYsDzD8SZpqZPdf6XXxVpFKPubpP8ZeI7zxh3RVt-9O3RypcJjvQPXwBgLP9Mf0ZcR8tcsIvzmNWLg4TeRWx8rT9KxSQyyvimKbtNJQi34esbsRU~mngFUgkKNtfNuoQ7sBTx7JOprKdjYPk5XZlkMR40ybbgPhMUmnAvBWFGmjwLLts8tnyO3v65~T8CAqNfieB-TKrFf26komkx8REATeTQ07gQ2~KusUDzCqQmssEy9YF4s7kKfMqk6r3t2KSJe1Fv4fQPyvMNal1UXmWp1gEqJDQrontYcdShLbOfPa4A__', 'width': 191, 'height': 200, 'annotations': [{'id': 'd4952f51-12c6-437c-bccf-56a832750567', 'type': 'segmentation', 'title': 'a', 'value': 'a', 'color': '#FFECDB', 'attributes': [], 'points': [[[38, 36, 3

## 7. データセットの全てのオブジェクトを取得

#### 使い方
1. オブジェクトが登録されたデータセットを用意する
2. パラメータのFASTLABEL_ACCESS_TOKENを設定
3. パラメータのDATASET_NAMEに用意したプロジェクトの名前を設定
4. 実行

#### パラメータ
1. FASTLABEL_ACCESS_TOKEN:　プラットフォーム上で発行するアクセストークン
2. DATASET_NAME:　データセットの名前

In [ ]:
import fastlabel
import os

FASTLABEL_ACCESS_TOKEN = "sample-token" # Replace with your access token
DATASET_NAME = "fastlabel-sample" # Replace with your dataset name

# Initialize client
os.environ["FASTLABEL_ACCESS_TOKEN"] = FASTLABEL_ACCESS_TOKEN
client = fastlabel.Client()


def get_all_dataset_objects(dataset_name: str, tags: list[str] = None):
    # Iterate pages until new tasks are empty.
    result = []
    offset = None

    print("Fetching dataset objects...")
    while True:
        datasets = client.get_dataset_objects(
            dataset=dataset_name, tags=tags, offset=offset
        )
        result.extend(datasets)

        if len(datasets) > 0:
            offset = len(result)
            print(offset, " fetched")
        else:
            break

    print("Fetching dataset objects finished!")
    print("Total dataset object count: ", len(result))
    return result


if __name__ == "__main__":
    print(get_all_dataset_objects(dataset_name=DATASET_NAME))


Fetching dataset objects...
2  fetched
Fetching dataset objects finished!
Total dataset object count:  2
[{'name': 'sample1.jpg', 'size': 1009370, 'width': 5184, 'height': 3888, 'tags': [], 'annotations': [{'id': '8bf32285-c99c-4dde-abc0-c6ffd548ccd1', 'type': 'bbox', 'title': 'animal', 'value': 'animal', 'points': [1387.3, 944.91, 2661.5, 3018.38], 'rotation': 0, 'confidenceScore': -1, 'color': '#4ad624', 'attributes': [], 'keypoints': []}], 'customMetadata': {}, 'createdAt': '2024-08-13T07:40:54.855Z', 'updatedAt': '2024-08-13T07:40:54.855Z'}, {'name': 'sample2.jpg', 'size': 375150, 'width': 2878, 'height': 1871, 'tags': [], 'annotations': [{'id': '49f5ef57-d4aa-4832-96d0-58911e0f817c', 'type': 'bbox', 'title': 'animal', 'value': 'animal', 'points': [1349.85, 167.97, 2394.59, 1750.39], 'rotation': 0, 'confidenceScore': -1, 'color': '#4ad624', 'attributes': [], 'keypoints': []}], 'customMetadata': {}, 'createdAt': '2024-08-13T07:40:57.052Z', 'updatedAt': '2024-08-13T07:40:57.052Z'}]
